In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
import re

In [7]:
input_file = "angst-data/input.txt"
file = open(input_file, "r") 
text = file.read() 
print('corpus length:', len(text))

corpus length: 160859


In [8]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
chars

total chars: 81


['\n',
 ' ',
 '!',
 '"',
 '$',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [9]:
## Chopping the text up into strings of 40 characters to train on
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 53607


In [10]:
## Vectorizing our sequences
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [11]:
## We're designing our model here- choosing the number of layers and such
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [12]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Showing the poetry quality at the end of each epoch
def on_epoch_end(epoch, logs):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        ## What we're doing is giving the bot 40 characters and asking
        ## it to predict the 41st character, over and over
        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [13]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

## Here's where we train the model on our data
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
53504/53607 [============================>.] - ETA: 0s - loss: 2.4600
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " 
a grain of sand. 
I once knew about tw"
 
a grain of sand. 
I once knew about twe the wall the wall the wall the wall the sall the wall the wall the sare the sare the wall the sall the wall the wall a sall with the warl the wall and the wall the wall where the wall the warl the wall the wall the wall where shall the wall the wall the wall the wall the warl the wall the the wall the wall the wall the wall the sare the warl shere the wall the wall the all the wall the wall the 
----- diversity: 0.5
----- Generating with seed: " 
a grain of sand. 
I once knew about tw"
 
a grain of sand. 
I once knew about twe beare the worloll burtaly and a lage and with reath lrour all the dall leverepurs and beither in hor the liver a sarers was was wat word the wis selly the deat surs the wall wher and the dad and a cartort the cans w

of his hand. What you know feed the Ins,
Whis the the reanch of the pangitaice a. xpen a partie, tell derfove, they graed never ag and the can by broth love
of fiee her cleac, and city Skind in sime spalains,

s.wwatfile
and in
my hear frostwere the prudritent whill and lewow
abligsing the fell tho eysed lasp
brigct
the kniven
to the time mimpers.
or the Tibloin
whitter and sem-of is The bes
me pedsool, as writch in the wo
----- diversity: 1.2
----- Generating with seed: "on the back 

of his hand. What you know"
on the back 

of his hand. What you know bust a wandeves all walks her left. rerear youther und asofie. readsthors on heile and men
redestschis; I can retle
cit 
some magged, wert to godeal ,
 Suivered like to the la, crinds
Aw agues brangin
wo refaller to : I'll will ne meach with a rud!

Is grover, sell-rewish in even the caust
youd you chiod else in
bach raches. Ow, and aOboout,
loof, ave Gomign of moundy peci
in Howild and the teway
53607/53607 [===========================

show scheduled for relearn to be was the way the way the track of the way the way the way the same me house and the read, and the same in the dark and a man, cannot walk and the way the said the dark and a stink of the dark and a strenter to the way the way the says the stars.
And the way the dreat clester to the way the way the way the way the said the dark and anyone.

And the way the way the way the way the way in th
----- diversity: 0.5
----- Generating with seed: "it the ramparts,
show scheduled for rele"
it the ramparts,
show scheduled for relesh to still and his stink his
and an and the sever to the part water to day the said the bark and in the street.

Is was contorded tere and the disame the pain.

I want on the way them weld ie the light
 untire home the lind a streating a strink our with a trug rear and disap
and the his hand the water the way the hanks and been the lind
in the dark had peen the dream
the disappeers and sendish to
----- diversity: 1.0
----- Generating with 

and dows the choughes? I stole, girst  come. To their man ancast
there,
it last comple blush in a lose capeles, of withhamed youffer,
how own unden I not wake on just miles
glassed with its the cace of her to the mathres
congers. The back against on write its grines
she stull be fift deTrursting with the Mofo
----- diversity: 1.2
----- Generating with seed: ".

have been young, and young

They left"
.

have been young, and young

They left ilturcines
a guis and criiled outsed fust,
 long rocing I  right park
tcem, sllew irbampored
soop
to you gealy with you, me like heagh borntoant of you hearne. 
1elw ipsupsiad through I need
her your with the deadher Affour
nea bath their changuymaget. No whan Anorrnighting
and before in bleady aclogs, that let the spart.
As my cast shift-of pira;
anyosders, iffooming cood. He stuff to the backnt
53607/53607 [==============================] - 94s 2ms/step - loss: 1.4575
Epoch 12/60
53504/53607 [============================>.] - ETA: 0s - loss: 1.4336

and the window and the window and the was the will be and the world.

Then I have the beauty with a string
and reserfic as not and the will be a dinight of the way they rewaSt
with a strings and burning the will be be had
to be the will be the store.

The stream that the stars of a strings from the charget

They sounk as if his she lived them I all

I say they sout has the reaung,
----- diversity: 0.5
----- Generating with seed: "bility zero;
 the air is hot as blood, t"
bility zero;
 the air is hot as blood, there was a chester, replar.

I am stor if the stream that was he was
porinist of the mise thought of while she lost and a strinks
and reself pigsing shore the light to fest and my and then where in the fities,
and the fill we was
condist which the but she life
whos it it was the thing, resert cast she was
beter the will be the winders.

A man be whit frow the disappers and shadow 
and we close wit
----- diversity: 1.0
----- Generating with seed: "bility zero;
 the air is hot as b

...

And the the fellies and body
as the stoes of the gloved the way it never
dain to the field, spoin of the world.

And the sime a morthing and stare the seash of a strange,

If I leve street, the water the seash would be i
----- diversity: 0.5
----- Generating with seed: " at a bird.
For every loved child, a chi"
 at a bird.
For every loved child, a child mother over outhiod his store
and the saybettomes to beting and be child,
net you all as nothing and caller telsh of the recourn
 aschan sentern a mud now would not were renk to notsersk in the the country and and body and how he dirc and furthing to hear would someond
which of a suddering and the walk in the wating beneary breaking in the world.
 Sound a strang sometoust 
a creater and calive 
----- diversity: 1.0
----- Generating with seed: " at a bird.
For every loved child, a chi"
 at a bird.
For every loved child, a child spows love
end show they tot welled ject.
 rea vuiling the arking to cat the carmon in would nish a stane.
Than

 the ion bene stims of undicaped from my ow
fireng bike goess the unkiss ons.
The turle he mese from
----- diversity: 1.2
----- Generating with seed: "er word for stasis:
you draw lines,
they"
er word for stasis:
you draw lines,
they stoes wither you feirm tety,
 palt yourselely like ourhed that oft.
shes 
watter, ladperice it, of tower. jest right on the aive
was he defirentaFblight

he ave cloura alming and little, teria' mpantish
fisebed a mouvible of here by the bivinigdeblocoout my casset cars.

He betreichan ceileved and be changer bedrenthis las.


ary admlinter.   wert ving
soiched not ben reels in ungl
and disgurathe
53607/53607 [==============================] - 86s 2ms/step - loss: 1.6175
Epoch 48/60
53504/53607 [============================>.] - ETA: 0s - loss: 1.6739
----- Generating text after Epoch: 47
----- diversity: 0.2
----- Generating with seed: "s,
though it has that effect on you, som"
s,
though it has that effect on you, someond




fith home shadow what was prop

worth the since the fied and the since her the since the secking a dogale.
The took the song the arrosschas ot and the seny


Iree the song mile the senduling and sheep out be the sound a sother to me be and wall she dreep
----- diversity: 0.5
----- Generating with seed: "rmed by his coming. He was silent
mute a"
rmed by his coming. He was silent
mute and in the frog the doull a soft to fiendehe
The dodies and the wore but mand its.

the straited in the shoe the smelletion was the never
who the since the dren squith and the corner.

If a lost be us ot the to good the falle awourd on ho the falls

Some and the hand as to fore the some
had in the brost on the drene,
deain and bun alow the since meek and relmore the are slide the sinderne.
The will
----- diversity: 1.0
----- Generating with seed: "rmed by his coming. He was silent
mute a"
rmed by his coming. He was silent
mute ac us to the reach done.
Thads dimberiness a smite the hangs is sheemfe-ms, 
That it worge what how of wild for f

de the stis proan shakes not that haspe wasown., cat indoy burs
ladk clomermbnows

thet as arym sleWith the catceerod the 
----- diversity: 1.2
----- Generating with seed: " to its underneath, tense
As a stream of"
 to its underneath, tense
As a stream of dremerireadmor perin gurped I mitss
losby meand.
Theetergherng
whe prosse tardks
Oy thise has melhnech,
a muth this and oven wasps yoors.
To rnoinfis wai r itthe.

 hise w en


So tind Indrks gow exrebtnaniarce
gidbwene oncnes to theegs forloverevotsd.
I jofofre-dipoeyceregrofsous the and ame ewantutryind
onlave on vecuons.
utrig as bys

Wo sells my awow.

mixurce redraalhitiy,
Mam mats cheawiten
53607/53607 [==============================] - 85s 2ms/step - loss: 2.1819
Epoch 55/60
53504/53607 [============================>.] - ETA: 0s - loss: 2.2728
----- Generating text after Epoch: 54
----- diversity: 0.2
----- Generating with seed: " the United States: 
in tidy shrines, sc"
 the United States: 
in tidy shrines, scand the the his t

the then lo and lok the sous tron the soreI theuth the waikit the the the the then whe anded the the the lout the thenglyor p with on the shast abor the mous outher the sou the the sou the the  touge the thent the ste the the blewing sha the the be with the whe the lon wate the xopee t has the sely andoundes the moug the singed the will to t
----- diversity: 0.5
----- Generating with seed: "d
 Back across the blackness, an insiste"
d
 Back across the blackness, an insiste gh the was roont herofron the wai pap aad.
o to tron  and in eate whing the bame I then sneith, way rdow and meree.
 a the mep  mone,
winh the thes ting at the shor  wing befor 
The wold Iretroblote bep bed an so thal ie ongale win it an the me ch abik ol toun the chep.
who chourt sterer in 

The dobured to camar to me, forte  wa th bi and torto. w
on Irs and the walk ing offor 
the drethen tall 
----- diversity: 1.0
----- Generating with seed: "d
 Back across the blackness, an insiste"
d
 Back across the blackness, a

In [14]:
model.save('angst1.h5')

In [25]:
## Here is where you can actually use the model on a given piece of text!

for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = 'There once was a cool guy named Cam.....'
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(800):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

----- diversity: 0.2
----- Generating with seed: "There once was a cool guy named Cam....."
There once was a cool guy named Cam......
 the bestorment the stream and the street 
The warm her back and the common of the distant 
The we've we have seen the sun was a strong, 
And when the country of the shadows and the common of the wood.
The warm the shakened of the seasong and the sea
and brother was the same who come, 
And the said west the sun was the stream and the finged the sea, 
And the sun was a guess and the cornial streams and the sun for the stare 
The bed the sun was the morning and the sun is not my spitted and the common of the distant 
The field the mountains of the colored by the work,
and the sun will to the sun was a stand 
The said, and the water of the sea, 
And born the sun was the sun is not flowers of the morning of the work,
the seating the room of the stone of the works
and the warmied and the way 
----- diversity: 0.5
----- Generating with seed: "There once was a 

/home/Randus/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


Beauth like the small greet, 
The resount of the shadows the conkering to my many old trees,
the bold had the mountains of every pain 
Who was the heart he says, and the rain and brown 
The pease and the sun the way the coon light of the boat.
And for the bones of bedian from a shakes the sea, 
See a callar lived it see the wain down to the wells of it,
angual of the columf in the work,
an of the flack on the flowers of the woods
The sun they said we should be and is well worsed or the pain and his rise and soul. 
And flass, or by the turfees of day of the sea
----- diversity: 1.0
----- Generating with seed: "There once was a cool guy named Cam....."
There once was a cool guy named Cam.....my pourds though to charded,
manianing we, greit is theory.
Soldy a towning, whan immortain neat-angine flower
wabun young mortale at onatio tongue use again mudes years, hie drawnce
There, in this lights that since I waking friend . .
For let many lockt trees they closer mort greetic blue, 
With str